In [2]:
from datetime import datetime, date, timedelta
import functools
from types import MappingProxyType

import numpy as np
import pandas as pd
import json
from scipy import interpolate
import seaborn as sns; sns.set()
import pathlib
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from scipy.interpolate import InterpolatedUnivariateSpline
from sklearn.linear_model import RANSACRegressor
from shapely.ops import nearest_points
from abc import ABC, abstractmethod, abstractproperty
from sklearn.cluster import DBSCAN
from matplotlib import pyplot as plt
from ipywidgets import widgets, interact, interactive_output, fixed
import folium
import ipyleaflet

plt.ioff()

pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", 100)
pd.set_option("display.float_format", "{:,.6f}".format)

# List files name contains bus info
list_bus_info_file_names = [
    "stops.json", "paths.json",
    "vars.json", "trips.json", "timetables.json",
    "routes.json"
]

PREDICTION_PATH = r"/data/ebms/feed/prediction"
PREDICTION_COLUMN_HEADERS = (
    "REQUESTTIME", "STOPID", "ROUTEID", 
    "ROUTEVARID", "BUSID", "LOCTIMESTAMP", 
    "DISTANCE", "SPEED",
    "TIMETOSTOP", "UNAMED",
)
BUSINFO_PATH = r"/data/ebms/feed/businfo"
DATE_TO_LOAD = "20200316"  # Date to analysis

In [3]:
%run ../notebook/func_nb.ipynb

In [4]:
stops_org, pred_org = get_data_processed(get_date_from_str(DATE_TO_LOAD))

stops_df = stops_org.copy()
pred_df = pred_org.copy()

stops_df["DELTADIS"] = stops_df.apply(
    lambda x: x["SPOINT"].distance(x["NEARESTPOINT"]),
    axis=1,
)

stops_df["DTDISORIG"] = stops_df.DELTADIS.copy()
stops_df["DELTADIS"] = stops_df.DELTADIS.copy() * 111139

stops_df.head()

Get bus info.
Get data from /data/ebms/feed/businfo/2020/03/16/routes.json
Get data from /data/ebms/feed/businfo/2020/03/16/timetables.json
Get data from /data/ebms/feed/businfo/2020/03/16/trips.json
Get data from /data/ebms/feed/businfo/2020/03/16/vars.json
Get data from /data/ebms/feed/businfo/2020/03/16/paths.json
Get data from /data/ebms/feed/businfo/2020/03/16/stops.json
Get log data.
/data/ebms/feed/prediction/2020_03_16.csv
Ranking stops sequences.
Get Ratio distance for each stops.


,ROUTEID,ROUTEVARID,STOPID,CODE,NAME,STOPTYPE,ZONE,WARD,ADDRESSNO,STREET,SUPPORTDISABILITY,STATUS,LNG,LAT,SEARCH,ROUTES,LINESTRING,SPOINT,DISTANCE,NEARESTPOINT,RANK,MAXRANK,SPOINTFSPOINT,SPOINTLSPOINT,STOPS_LINESTRING,STOPSDISTANCE,ALLSTOPSDISTANCE,RATIOSTOPSDISTANCE,DELTADIS,DTDISORIG
0,1,1,33,BX 06,Công Trường Mê Linh,Bến xe,Quận 1,None,Công trường Mê Linh,Thi Sách,,Đang khai thác,106.705856,10.776790,CTML CtML TS,"01, 02","LINESTRING (106.70585632 10.77678967, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.706...",POINT (106.705856 10.77679),0.000000,POINT (106.70585632 10.77678967),1.000000,29.000000,POINT (106.705856 10.77679),POINT (106.6525650024414 10.75125312805176),"LINESTRING (106.705856 10.77679, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.70605469...",0.000000,0.076539,0.000000,0.051088,0.000000
1,1,1,79,Q1 031,Bến Bạch Đằng,Trụ dừng,Quận 1,None,21,Tôn Đức Thắng,,Đang khai thác,106.706359,10.773299,BBD 21 TDT,"01, 02, 03, 120, 19, 45, 53, 56, 61-6, 88, D1, DL01","LINESTRING (106.70585632 10.77678967, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.706...",POINT (106.706359 10.773299),0.004051,POINT (106.7063598425827 10.77329894636785),2.000000,29.000000,POINT (106.705856 10.77679),POINT (106.6525650024414 10.75125312805176),"LINESTRING (106.705856 10.77679, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.70605469...",0.004051,0.076539,0.052928,0.093833,0.000001
2,1,1,76,Q1 020,Cục Hải Quan Thành Phố,Nhà chờ,Quận 1,None,2-4,Hàm Nghi,,Đang khai thác,106.705549,10.770885,CHQTP 2-4 HN,"01, 02, 03, 123, 124, 19, 45, 56, 61-6, 88, D1","LINESTRING (106.70585632 10.77678967, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.706...",POINT (106.705549 10.770885),0.007071,POINT (106.7055490282718 10.77088557442607),3.000000,29.000000,POINT (106.705856 10.77679),POINT (106.6525650024414 10.75125312805176),"LINESTRING (106.705856 10.77679, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.70605469...",0.007071,0.076539,0.092389,0.063918,0.000001
3,1,1,3691,Q1 193,Chợ Cũ,Nhà chờ,Quận 1,None,84,Hàm Nghi,,Đang khai thác,106.703108,10.771006,CC 84 HN,"01, 02, 03, 19, 45, 56, 86, 88","LINESTRING (106.70585632 10.77678967, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.706...",POINT (106.703108 10.771006),0.009515,POINT (106.703107990596 10.77100568168236),4.000000,29.000000,POINT (106.705856 10.77679),POINT (106.6525650024414 10.75125312805176),"LINESTRING (106.705856 10.77679, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.70605469...",0.009515,0.076539,0.124320,0.035393,0.000000
4,1,1,2028,Q1 194,Trường Cao Thắng,Nhà chờ,Quận 1,None,Trường Cao Thắng,Hàm Nghi,,Đang khai thác,106.701853,10.771043,TCT TCT HN,"01, 02, 03, 102, 11, 19, 20, 34, 38, 39, 45, 56, 75, 86, 88","LINESTRING (106.70585632 10.77678967, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.706...",POINT (106.701853 10.771043),0.010770,POINT (106.701852992847 10.77104275787594),5.000000,29.000000,POINT (106.705856 10.77679),POINT (106.6525650024414 10.75125312805176),"LINESTRING (106.705856 10.77679, 106.7062912 10.77581787, 106.70614624 10.77576542, 106.70605469...",0.010771,0.076539,0.140724,0.026921,0.000000


In [5]:
stops_df["LABEL"] = (
    stops_df["STOPTYPE"]
    + " "
    + stops_df["NAME"] 
    + " - đường "
    + stops_df["STREET"]
    + ", "
    + stops_df["ZONE"]
)

stops_df_select = stops_df.groupby(["ROUTEID", "ROUTEVARID"]).agg(
    SPOINT=("SPOINT", list),
    LINESTRING=("LINESTRING", "first"),
    NEARESTPOINT=("NEARESTPOINT", list),
    LABEL=("LABEL", list),
    STOPTYPE=("STOPTYPE", list),
    NAME=("NAME", list),
    STREET=("STREET", list),
    ZONE=("ZONE", list),
    DELTADIS=("DELTADIS", list),
    LATMAX=("LAT", "max"),
    LATMIN=("LAT", "min"),
    LNGMAX=("LNG", "max"),
    LNGMIN=("LNG", "min"),
).reset_index()

stops_df_select.head()

def add_marker(
    fl_map, points, 
    label, color="blue", icon="info-sign", 
    prefix_icon="glyphicon", opacity=0.9,
    feature_group=None
):
    for p, l in zip(points, label):
        po_lng = p.coords.xy[0][0]
        po_lat = p.coords.xy[1][0]
        marker = folium.Marker(
            icon=folium.Icon(icon=icon, color=color, prefix=prefix_icon),
            location=[po_lat, po_lng],
            tooltip=folium.Tooltip(l),
            opacity=opacity,
        ).add_to(fl_map)
        
        if feature_group:
            feature_group.add_child(marker)

def plot_folium_map(stops, margin=1, addition_map=None):
    if addition_map is None:
        fl_map = folium.Map(
            tiles=None,
            zoom_start=9,
        )
    else:
        fl_map = addition_map
    
    # init feature group
    folium.TileLayer(tiles="OpenStreetMap", name="Open Street Map").add_to(fl_map)
    folium.TileLayer(tiles="Stamen Terrain", name="Stamen Terrain").add_to(fl_map)
    folium.TileLayer(tiles="Stamen Watercolor", name="Stamen Watercolor").add_to(fl_map)
    folium.TileLayer(tiles="Stamen Toner", name="Stamen Toner").add_to(fl_map)
    folium.TileLayer(tiles="CartoDB positron", name="CartoDB Positron").add_to(fl_map)
    folium.TileLayer(tiles="CartoDB dark_matter", name="CartoDB Dark Matter").add_to(fl_map)
    
    # add group to map
    nearest_group = folium.FeatureGroup(name="Nearest points").add_to(fl_map)
    stops_group = folium.FeatureGroup(name="Stops", show=False).add_to(fl_map)
    line_group = folium.FeatureGroup(name="Bus Path").add_to(fl_map)
    
    # draw marker
    add_marker(
        fl_map, stops.SPOINT.values[0], stops.LABEL.values[0], 
        icon="fa-street-view", prefix_icon="fa", 
        feature_group=stops_group, color="darkblue"
    )
    add_marker(
        fl_map, stops.NEARESTPOINT.values[0], stops.LABEL.values[0], 
        icon="fa-bus", prefix_icon="fa", feature_group=nearest_group, 
        color="darkred"
    )
    
    # draw busline and add busline to feature group
    lines = [tuple(x) for x in list(stops.LINESTRING.values[0].coords)]
    lines = [t[::-1] for t in lines]
    bus_line = folium.PolyLine(locations=lines, color="green", weight=5).add_to(fl_map)
    line_group.add_child(bus_line)
    folium.LayerControl(collapsed=False).add_to(fl_map)
    
    # Zoom fit to street
    fl_map.fit_bounds(
        [
            (stops.LATMIN.values[0] * margin, stops.LNGMIN.values[0] * margin),
            (stops.LATMAX.values[0] * margin, stops.LNGMAX.values[0] * margin), 
        ]
    )
    
    return fl_map

test_route = stops_df_select[(stops_df_select.ROUTEID==1) & (stops_df_select.ROUTEVARID==1)].copy()
plot_folium_map(test_route)

In [6]:
list_route_df = stops_df[["ROUTEID", "ROUTEVARID"]].drop_duplicates().copy()
list_route = list_route_df.groupby(["ROUTEID"]).agg(list).to_dict()["ROUTEVARID"]
list_route_id = list(list_route.keys())

list_route_df = stops_df[["ROUTEID", "ROUTEVARID"]].drop_duplicates().copy()
list_route = list_route_df.groupby(["ROUTEID"]).agg(list).to_dict()["ROUTEVARID"]
list_route_id = list(list_route.keys())

wdg_route_id = widgets.Dropdown(options=list_route_id, description="Route Id: ", index=0)
wdg_route_var_id = widgets.Dropdown(options=list_route[list_route_id[0]], description="Route Var Id: ", index=1)
wdg_ui = widgets.HBox([wdg_route_id, wdg_route_var_id])


def get_list_route_var(*args):
    wdg_route_var_id.options = list_route[wdg_route_id.value]


def plot_route_folium_wrapper(stops):
    def _inner(route_id, route_var_id):     
        try:
            data_df = stops[
                (stops.ROUTEID==route_id) 
                & (stops.ROUTEVARID==route_var_id)
            ].copy()
        except Exception as e:
            data_df = pd.DataFrame()
        
        if not data_df.empty:
            map_it = plot_folium_map(data_df)
            display(map_it)
            map_it.save("../html_map/route_3_5.html")
        else:
            display("NOT DATA")

    return _inner


wdg_route_id.observe(get_list_route_var, "value")

out = widgets.interactive_output(
    plot_route_folium_wrapper(stops_df_select), {
        'route_id': wdg_route_id,
        'route_var_id': wdg_route_var_id,
    },
)

display(wdg_ui, out)

Output()

# Using IpyLeafLet

In [13]:
from ipyleaflet import AntPath, Map

m = Map(center=(51.332, 6.853), zoom=10)

ant_path = AntPath(
    locations=[
        [51.185, 6.773], [51.182, 6.752], [51.185, 6.733], [51.194, 6.729],
        [51.205, 6.732], [51.219, 6.723], [51.224, 6.723], [51.227, 6.728],
        [51.228, 6.734], [51.226, 6.742], [51.221, 6.752], [51.221, 6.758],
        [51.224, 6.765], [51.230, 6.768], [51.239, 6.765], [51.246, 6.758],
        [51.252, 6.745], [51.257, 6.724], [51.262, 6.711], [51.271, 6.701],
        [51.276, 6.702], [51.283, 6.710], [51.297, 6.725], [51.304, 6.732],
        [51.312, 6.735], [51.320, 6.734], [51.326, 6.726], [51.334, 6.713],
        [51.340, 6.696], [51.344, 6.678], [51.349, 6.662], [51.354, 6.655],
        [51.360, 6.655], [51.366, 6.662], [51.369, 6.675], [51.373, 6.704],
        [51.376, 6.715], [51.385, 6.732], [51.394, 6.741], [51.402, 6.743],
        [51.411, 6.742], [51.420, 6.733], [51.429, 6.718], [51.439, 6.711],
        [51.448, 6.716], [51.456, 6.724], [51.466, 6.719], [51.469, 6.713],
        [51.470, 6.701], [51.473, 6.686], [51.479, 6.680], [51.484, 6.680],
        [51.489, 6.685], [51.493, 6.700], [51.497, 6.714]
    ],
    dash_array=[1, 10],
    delay=1000,
    color='#7590ba',
    pulse_color='#3f6fba'
)

m.add_layer(ant_path)
m

Map(center=[51.332, 6.853], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…